In [4]:
import json

# Load JSON data
with open("filtered_data.json", "r") as f:
    bird_data = json.load(f)


import os

# ✅ Extract Bird Species Labels
labels = list(bird_data.keys())  # List all bird species
label_mapping = {species: idx for idx, species in enumerate(labels)}  # Assign numeric labels
reverse_label_mapping = {idx: species for species, idx in label_mapping.items()}  # Label → Bird

print(f"✅ Bird Species: {labels}")
print(f"✅ Label Mapping: {label_mapping}")

✅ Bird Species: ['American Robin', "Bewick's Wren", 'Northern Cardinal', 'Northern Mockingbird', 'Song Sparrow']
✅ Label Mapping: {'American Robin': 0, "Bewick's Wren": 1, 'Northern Cardinal': 2, 'Northern Mockingbird': 3, 'Song Sparrow': 4}


In [30]:
import json
import os
import random
import torch
import librosa
import numpy as np
import torch.nn.functional as F

# ✅ Load JSON Data (Bird Species & File Info)
with open("filtered_data.json", "r") as f:
    bird_data = json.load(f)

# ✅ Extract Bird Species Labels
labels = list(bird_data.keys())  
label_mapping = {species: idx for idx, species in enumerate(labels)}  
reverse_label_mapping = {idx: species for species, idx in label_mapping.items()}  

print(f"✅ Bird Species: {labels}")
print(f"✅ Label Mapping: {label_mapping}")

# ✅ Define Model Path
MODEL_PATH = "best_model.pth"  # Ensure this file exists

# ✅ Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Import CNN Model
from cnn import ImprovedBirdSoundCNN  # Ensure cnn_model.py exists

# ✅ Define Model (Ensure num_classes matches training)
num_classes = len(labels)  # Automatically set based on JSON
model = ImprovedBirdSoundCNN(num_classes=num_classes).to(device)

# ✅ Load Model Weights (Handle missing keys properly)
try:
    state_dict = torch.load(MODEL_PATH, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Model loading failed: {e}")
    exit()

model.eval()  # Set model to evaluation mode

# ✅ Create Dictionary {filename: label}
file_to_label = {bird["filename"]: bird["name"] for bird_list in bird_data.values() for bird in bird_list}

# ✅ Pick a Random File for Testing
random_file, true_label = random.choice(list(file_to_label.items()))
file_path = os.path.join("wavfiles", random_file)  

print(f"🎲 Selected File: {random_file}")
print(f"🏷️ True Label: {true_label}")

# ✅ Load and Process the Audio File
y, sr = librosa.load(file_path, sr=None)

# ✅ Convert to Mel Spectrogram
mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

# ✅ Convert to PyTorch Tensor & Ensure Correct Shape
mel_tensor = torch.tensor(mel_spec_db, dtype=torch.float32).unsqueeze(0)  # Shape (1, 128, T)

# ✅ Resize to (1, 1, 128, 128) to Match Model Input
mel_tensor = F.interpolate(mel_tensor.unsqueeze(0), size=(128, 128), mode="bilinear", align_corners=False)
mel_tensor = mel_tensor.to(device)  

# ✅ Run Model Prediction
with torch.no_grad():
    output = model(mel_tensor)
    predicted_label = output.argmax(dim=1).item()
    pred_class = reverse_label_mapping.get(predicted_label, "Unknown")  # Handle missing labels

# ✅ Print Results
print(f"\n🔍 File: {random_file}")
print(f"🤖 Predicted Class: {pred_class}")
print(f"🏷️ True Class: {true_label} ({'✅ Correct' if pred_class == true_label else '❌ Incorrect'})")


✅ Bird Species: ['American Robin', "Bewick's Wren", 'Northern Cardinal', 'Northern Mockingbird', 'Song Sparrow']
✅ Label Mapping: {'American Robin': 0, "Bewick's Wren": 1, 'Northern Cardinal': 2, 'Northern Mockingbird': 3, 'Song Sparrow': 4}
✅ Model loaded successfully!
🎲 Selected File: 563619-8.wav
🏷️ True Label: Northern Cardinal

🔍 File: 563619-8.wav
🤖 Predicted Class: Northern Cardinal
🏷️ True Class: Northern Cardinal (✅ Correct)
